In [1]:
from datasets import load_dataset
from training_datasets.GLUEDataset import GLUEDataset

cola = load_dataset('glue','cola')
# print(cola)
# sst2 = load_dataset('glue','sst2')
# print(sst2)
mrpc = load_dataset('glue', 'mrpc')
# print(mrpc)
pt_cola = GLUEDataset([cola, mrpc], 'train')

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1032.32it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1448.14it/s]


In [2]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

# sampler = FewShotEpisodeSampler(pt_cola, kShot=2, nWay=4, shuffle=True)
# train_data_loader = data.DataLoader(
#     pt_cola,
#     batch_sampler=sampler,
#     num_workers=1,
# )

# data, targets = next(iter(train_data_loader))

# for i in range(5):
#     data, targets = next(iter(train_data_loader))
#     print(data)
#     print(targets)

train_protomaml_sampler = FewShotEpisodeBatchSampler(pt_cola, kShot=3, nWay=4, batchSize=8, shuffle=True)
train_protomaml_loader = data.DataLoader(
    pt_cola, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=1
)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)

In [3]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [4]:
from training.models.ProtoFOMAML import ProtoFOMAML

pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relation

losses are tensor([0.5351, 0.6120, 0.4798, 0.4584, 0.5762, 0.5325],
       grad_fn=<NllLossBackward0>) and loss is 3.193829298019409
losses are tensor([0.5298, 0.6090, 0.4751, 0.4469, 0.5679, 0.5224],
       grad_fn=<NllLossBackward0>) and loss is 3.1511826515197754
losses are tensor([0.5244, 0.6058, 0.4702, 0.4360, 0.5601, 0.5128],
       grad_fn=<NllLossBackward0>) and loss is 3.109300136566162
losses are tensor([0.5187, 0.6022, 0.4651, 0.4258, 0.5529, 0.5038],
       grad_fn=<NllLossBackward0>) and loss is 3.068528890609741
losses are tensor([0.5131, 0.5984, 0.4601, 0.4160, 0.5458, 0.4951],
       grad_fn=<NllLossBackward0>) and loss is 3.0284793376922607
outer loop losses are tensor([0.5269, 0.6717, 0.6292, 0.6731, 0.4846, 0.6795],
       grad_fn=<NllLossBackward0>) and loss is 3.665151834487915
local param grad is tensor([[ 4.4154e-05, -7.7068e-05,  3.1897e-05,  ...,  1.2258e-04,
          5.8701e-05, -4.2431e-05],
        [-5.7509e-05, -2.4430e-05, -3.0660e-07,  ..., -1.1096e-05,

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relation

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [5]:
from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=2)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

# val_protomaml_loader = data.DataLoader(
#     None, batch_sampler=val_protomaml_sampler, num_workers=1
# )


In [6]:
for i in range(1):
    batch = next(iter(val_protomaml_loader))
#     print(batch)
    for episode_i in range(len(batch[0])):
        data, labels = batch[0][episode_i], batch[1][episode_i]
        supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2]
        print(supportLabels)

[0, 0, 1, 1]
[2, 4, 3, 3, 2, 4]
[7, 5, 7, 6, 5, 6]
[10, 10, 8, 9, 8, 9]
[11, 12, 12, 13, 13, 11]
[15, 15, 14, 14]
[17, 16, 16, 17]
[18, 19, 18, 19]
